In [18]:
import logging
logging.basicConfig(level=logging.ERROR)
import nest_asyncio
nest_asyncio.apply()

import numpy as np

from page_reader import ConcurrentWrapper, SimplePageReader
from rerankers import Reranker
from rewriters import QwenRewriter
from search_engines.bing import BingRequest, BingTextSearch
from splitters import split_search_records

def format_information(search_records):
    knowledge_template = """
    <Search Record>
    Title: {title}
    URL: {url}
    Content: {content}
    </Search Record>
    """.strip()

    return "\n\n".join([knowledge_template.format(
        title = s.title.strip(),
        url = s.url.strip(), 
        content = s.content.strip()) 
    for s in search_records])


def execute_search_engine(q:str, topn:int = 5):
    # 构建请求
    text_req = BingRequest.chinese(q)
    
    # 执行搜索
    rewriter = QwenRewriter()
    text_searcher = BingTextSearch(rewriter = rewriter)
    search_records = text_searcher.search_normalize(text_req)
    
    # 详情爬虫
    page_reader = ConcurrentWrapper(reader = SimplePageReader())
    urls = [r.url for r in search_records]
    contents = page_reader.read(urls)
    for i, s in enumerate(search_records):
        s.set_content(contents[i])
       
    # 内容分割 
    chunked_records = split_search_records(search_records)
    
    # 内容精排
    reranker = Reranker()
    
    docs = [s.content for s in chunked_records]
    scores = reranker.rerank(q, docs)
    top_records = np.array(chunked_records)[np.argsort(scores)[::-1]][:topn].tolist()
        
    return top_records


def search_tool(query:str, topn:int = 5) -> str:
    """Search microsoft bing against a query, rerank results and return topn results as a knowledge string."""
    search_records = execute_search_engine(query, topn)
    return format_information(search_records)

In [19]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama

bing_tool = FunctionTool.from_defaults(
    search_tool
)

agent = ReActAgent.from_tools(
    tools = [bing_tool], 
    llm=Ollama(model="qwen2.5:14b"),
    verbose=True
)

In [21]:
agent.chat(message = "新疆有一个叫做五家渠的地方么？")

> Running step 1176a73b-c1cc-4f30-b428-c77f2678c3a6. Step input: 新疆有一个叫做五家渠的地方么？
Thought: The current language of the user is: zh. I need to use a tool to answer the question.
Action: search_tool
Action Input: {'query': '新疆 五家渠', 'topn': 5}
Using query:  五家渠 新疆 旅游景点
Reranking 71 documents...
Reranking taking 0.74 seconds
Observation: <Search Record>
    Title: 新疆五家渠的哪些景点值得安利给游客 (新疆五家渠旅游景点)
    URL: https://www.liantu.cn/gonglue/k96669.html
    Content: 联途 > 旅游攻略 > 新疆五家渠的哪些景点值得安利给游客(新疆五家渠旅游景点)
新疆五家渠是位于中国新疆维吾尔自治区的一个城市，虽然不像其他一些新疆城市那样闻名遐迩，但这里依然拥有独特的自然风光和文化遗产。以下是一些值得游客前往的五家渠景点：  
这是一个集观光、休闲、教育和科研于一体的综合性湿地公园。公园内有丰富的水生植物和鸟类资源，是观鸟爱好者的理想之地。在这里，游客可以体验宁静的自然风光，感受大自然的恬静与和谐。
    </Search Record>

<Search Record>
    Title: 2024五家渠旅游攻略,五家渠自由行攻略,马蜂窝五家渠出游 ...
    URL: https://www.mafengwo.cn/travel-scenic-spot/mafengwo/32748.html
    Content: 五家渠市位于新疆维吾尔自治区中部天山山脉北麓、准噶尔盆地东南缘，与昌吉市、乌鲁木齐市相接。
该市是新疆维吾尔自治区天山北坡经济腹心地带，也是从乌鲁木齐到古尔班... 更多>> wangpinga
“太偏了，打车一点都不方便，还有就是周围没有餐馆，酒店餐厅服务很好，因为早上赶飞机，特意吩咐餐厅阿姨帮我提前准备了一份早餐，感动

AgentChatResponse(response='新疆确实有一个叫做五家渠的地方。它位于中国新疆维吾尔自治区中部天山山脉北麓、准噶尔盆地东南缘，是一个集观光、休闲、教育和科研于一体的综合性城市。五家渠拥有丰富的自然风光和文化遗产，如青格达湖风景区、古尔班通古特沙漠等旅游景点都值得一游。\n\n您有其他问题吗？', sources=[ToolOutput(content='<Search Record>\n    Title: 新疆五家渠的哪些景点值得安利给游客 (新疆五家渠旅游景点)\n    URL: https://www.liantu.cn/gonglue/k96669.html\n    Content: 联途 > 旅游攻略 > 新疆五家渠的哪些景点值得安利给游客(新疆五家渠旅游景点)\n新疆五家渠是位于中国新疆维吾尔自治区的一个城市，虽然不像其他一些新疆城市那样闻名遐迩，但这里依然拥有独特的自然风光和文化遗产。以下是一些值得游客前往的五家渠景点：  \n这是一个集观光、休闲、教育和科研于一体的综合性湿地公园。公园内有丰富的水生植物和鸟类资源，是观鸟爱好者的理想之地。在这里，游客可以体验宁静的自然风光，感受大自然的恬静与和谐。\n    </Search Record>\n\n<Search Record>\n    Title: 2024五家渠旅游攻略,五家渠自由行攻略,马蜂窝五家渠出游 ...\n    URL: https://www.mafengwo.cn/travel-scenic-spot/mafengwo/32748.html\n    Content: 五家渠市位于新疆维吾尔自治区中部天山山脉北麓、准噶尔盆地东南缘，与昌吉市、乌鲁木齐市相接。\n该市是新疆维吾尔自治区天山北坡经济腹心地带，也是从乌鲁木齐到古尔班... 更多>> wangpinga\n“太偏了，打车一点都不方便，还有就是周围没有餐馆，酒店餐厅服务很好，因为早上赶飞机，特意吩咐餐厅阿姨帮我提前准备了一份早餐，感动。 永远不满..\n“已偕同家人入住五家渠迎宾馆好多次了（自2014年10.1小长假开始）。 本次由于只提前了一天预订，房间为1楼，临近停车场比较吵。 早餐一直觉...” 鹿徒\n“还可以，休闲逛逛的人挺多的。 鹿徒 “去看郁金香的时候去过，人还挺多的，烤鱼不错。 